# Market

In [33]:
symbols = 'SPY'
# symbols = ['SPY', 'GLD', 'SLV']

# Position size

In [38]:
from datetime import datetime, timedelta

from pandas.tseries.offsets import BDay
from quantopian.research import get_pricing

PRICE_SLIDING_WINDOW = 57

end_date = datetime.now()
start_date = end_date - BDay(PRICE_SLIDING_WINDOW)

prices = get_pricing(
    symbols,
    start_date=start_date,
    end_date=end_date,
    fields=[
        'price',
        'high',
        'low',
        'close_price'
    ]
)

prices = prices.dropna()

prices.head()

,price,high,low,close_price
2017-02-16 00:00:00+00:00,233.723,234.136,232.837,233.723
2017-02-17 00:00:00+00:00,234.042,234.052,232.917,234.042
2017-02-21 00:00:00+00:00,235.426,235.665,234.490,235.426
2017-02-22 00:00:00+00:00,235.267,235.515,234.809,235.267
2017-02-23 00:00:00+00:00,235.416,235.874,234.540,235.416


In [39]:
from talib import ATR

AVERAGE_TRUE_RANGE_SLIDING_WINDOW = 21
MOVING_AVERAGE = 20

volatility = ATR(
    prices.high[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    prices.low[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    prices.close_price[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    timeperiod=MOVING_AVERAGE
)[-1]

volatility

1.4894999999999954

In [40]:
DOLLARS_PER_POINT = 1

dollar_volatility = volatility * DOLLARS_PER_POINT

dollar_volatility

1.4894999999999954

In [41]:
CAPITAL_PERCENTAGE_PER_TRADE = 0.01
START_CAPITAL = 1000000
CURRENT_CAPITAL = 900000
CAPITAL_MULTIPLIER = 2

profit = CURRENT_CAPITAL - START_CAPITAL
available_capital = START_CAPITAL

if profit < 0:
    available_capital = START_CAPITAL + profit * CAPITAL_MULTIPLIER

capital_per_trade = available_capital * CAPITAL_PERCENTAGE_PER_TRADE
trade_size = capital_per_trade / dollar_volatility

int(trade_size)

5370

# Entry

In [55]:
TWENTY_DAY_BREAKOUT = 20
FIFTY_FIVE_DAY_BREAKOUT = 55

In [59]:
twenty_day_high = prices.high[-TWENTY_DAY_BREAKOUT:].max()

twenty_day_high

239.72

In [60]:
fifty_five_day_high = prices.high[-FIFTY_FIVE_DAY_BREAKOUT:].max()

fifty_five_day_high

239.72

In [62]:
twenty_day_low = prices.low[-TWENTY_DAY_BREAKOUT:].min()

twenty_day_low

232.55000000000001

In [63]:
fifty_five_day_low = prices.low[-FIFTY_FIVE_DAY_BREAKOUT:].min()

fifty_five_day_low

231.61000000000001

In [65]:
latest_price = prices.price[-1]

latest_price

239.72

In [85]:
def get_system_one_entry_signal():
    """
    Get 20 day breakout entry signal.
    
    Does not factor in outcome of previous entry signal.
    """
    if latest_price > twenty_day_high:
        print('System one: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < twenty_day_low:
        print('System one: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System one: No signal')
        
get_system_one_entry_signal()

System one: No signal


In [86]:
def get_system_two_entry_signal():
    """
    Get 55 day breakout entry signal.
    """
    if latest_price > fifty_five_day_high:
        print('System two: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < fifty_five_day_low:
        print('System two: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System two: No signal')
        
get_system_two_entry_signal()

System two: No signal


In [87]:
latest_price = 240

get_system_one_entry_signal()

System one: Long SPY 5370@240.00


In [88]:
latest_price = 232

get_system_one_entry_signal()

System one: Short SPY 5370@232.00


In [89]:
latest_price = 240

get_system_two_entry_signal()

System two: Long SPY 5370@240.00


In [92]:
latest_price = 231

get_system_two_entry_signal()

System two: Short SPY 5370@231.00


In [107]:
def add_unit():
    """
    Add to position if price exceeds threshold.
    """
    previous_fill_price = 240

    long_threshold = previous_fill_price + volatility / 2
    short_threshold = previous_fill_price - volatility / 2

    if latest_price > long_threshold:
        print('Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < short_threshold:
        print('Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('No signal')

In [108]:
latest_price = 241

add_unit()

Long SPY 5370@241.00


In [110]:
latest_price = 239

add_unit()

Short SPY 5370@239.00


In [112]:
latest_price = 239.5

add_unit()

No signal


# Stop

In [117]:
STOP_MULTIPLIER = 2

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

237.02100000000002

In [118]:
# Re-enter if price hits entry price after stopped out
STOP_MULTIPLIER = 0.5

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

239.25524999999999